In [1]:
!pip install ultralytics roboflow opencv-python-headless supervision pandas


In [2]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ultralytics import YOLO
import supervision as sv
from IPython.display import Video, display
import pandas as pd


def extract_frames_for_annotation(video_path, output_dir, frame_interval=30):
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        if frame_count % frame_interval == 0:
            frame_path = f"{output_dir}/frame_{saved_count:04d}.jpg"
            cv2.imwrite(frame_path, frame)
            saved_count += 1
            
        frame_count += 1
    
    cap.release()
    print(f"Extracted {saved_count} frames from {frame_count} total frames")
    return saved_count


extract_frames_for_annotation("Video.mp4", "frames_to_annotate", frame_interval=30)


Extracted 290 frames from 8674 total frames


290

In [4]:
def train_vehicle_detector(data_yaml_path, epochs=50, img_size=640):
    """
    Train YOLOv8 model for vehicle detection
    
    Args:
        data_yaml_path: Path to data.yaml file
        epochs: Number of training epochs
        img_size: Input image size
    """
    # Load a pre-trained YOLOv8 model
    model = YOLO('yolov8n.pt')  # nano version (fastest)
    # Alternative: 'yolov8s.pt' (small), 'yolov8m.pt' (medium)
    
    # Train the model
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=img_size,
        batch=16,
        name='vehicle_counter',
        patience=10,  # Early stopping
        save=True,
        plots=True,
        device=0  # Use 0 for GPU, 'cpu' for CPU
    )
    
    return model, results

model, results = train_vehicle_detector('My First Project.v2i.yolov8/data.yaml', epochs=50)

Ultralytics 8.3.226 🚀 Python-3.12.11 torch-2.8.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=My First Project.v2i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=vehicle_counter2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspe

[W1108 07:27:41.334218144 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.339390029 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.342493794 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.346954581 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.349244427 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.349751399 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.350594759 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.351140463 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.351662975 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:27:41.352347772 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6081.1±1415.6 MB/s, size: 373.8 KB)
train: Scanning /teamspace/studios/this_studio/My First Project.v2i.yolov8/train/labels.cache... 176 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 176/176 363.3Kit/s 0.0s
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3557.9±3225.9 MB/s, size: 359.1 KB)
val: Scanning /teamspace/studios/this_studio/My First Project.v2i.yolov8/valid/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 104.9Kit/s 0.0s
Plotting labels to /teamspace/studios/this_studio/runs/detect/vehicle_counter2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bia

[W1108 07:28:47.992302301 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.992937546 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.993201253 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.993493520 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.993687455 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.993867198 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.994070204 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.994255648 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.994445283 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.
[W1108 07:28:47.994598977 NNPACK.cpp:56] Could not initialize NNPACK! Reason: Unsupported hardware.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 9.9it/s 0.2s0.4s
                   all         50        861      0.907      0.819      0.929      0.823
Speed: 0.0ms preprocess, 0.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /teamspace/studios/this_studio/runs/detect/vehicle_counter2


In [5]:

class VehicleCounter:
    """Vehicle counting system with tracking"""
    
    def __init__(self, model_path, conf_threshold=0.25):
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        
    def process_video(self, input_video, output_video):
        """
        Process video and count vehicles
        
        Args:
            input_video: Path to input video
            output_video: Path to save output video
        """
        cap = cv2.VideoCapture(input_video)
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
        
        # Initialize tracker
        byte_tracker = sv.ByteTrack()
        
        # Annotation helpers
        box_annotator = sv.BoxAnnotator(thickness=2)
        label_annotator = sv.LabelAnnotator()
        
        frame_count = 0
        vehicle_counts = []
        tracked_ids = set()
        
        print(f"Processing {total_frames} frames...")
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run detection
            results = self.model(frame, conf=self.conf_threshold, verbose=False)[0]
            
            # Convert to supervision format
            detections = sv.Detections.from_ultralytics(results)
            
            # Update tracker
            detections = byte_tracker.update_with_detections(detections)
            
            # Count unique vehicles
            if detections.tracker_id is not None:
                for tracker_id in detections.tracker_id:
                    tracked_ids.add(tracker_id)
            
            current_count = len(detections)
            total_unique = len(tracked_ids)
            
            # Prepare labels
            labels = [
                f"ID:{tracker_id}" if tracker_id is not None else "New"
                for tracker_id in (detections.tracker_id if detections.tracker_id is not None else [None] * len(detections))
            ]
            
            # Annotate frame
            annotated_frame = frame.copy()
            annotated_frame = box_annotator.annotate(annotated_frame, detections)
            annotated_frame = label_annotator.annotate(annotated_frame, detections, labels)
            
            # Add count text
            cv2.putText(annotated_frame, f"Current Vehicles: {current_count}", 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"Total Unique: {total_unique}", 
                       (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, f"Frame: {frame_count}/{total_frames}", 
                       (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            out.write(annotated_frame)
            
            # Store data
            vehicle_counts.append({
                'frame': frame_count,
                'current_count': current_count,
                'total_unique': total_unique
            })
            
            frame_count += 1
            
            if frame_count % 30 == 0:
                print(f"Processed {frame_count}/{total_frames} frames")
        
        cap.release()
        out.release()
        
        # Save statistics
        df = pd.DataFrame(vehicle_counts)
        df.to_csv('vehicle_count_stats.csv', index=False)
        
        print(f"\nProcessing Complete!")
        print(f"Total unique vehicles tracked: {total_unique}")
        print(f"Average vehicles per frame: {df['current_count'].mean():.1f}")
        print(f"Max vehicles in frame: {df['current_count'].max()}")
        
        return df, total_unique

# Example usage (uncomment to run):
counter = VehicleCounter('runs/detect/vehicle_counter2/weights/best.pt')
stats_df, total_vehicles = counter.process_video('Video.mp4', 'output.mp4')


Processing 8674 frames...
Processed 30/8674 frames
Processed 60/8674 frames
Processed 90/8674 frames
Processed 120/8674 frames
Processed 150/8674 frames
Processed 180/8674 frames
Processed 210/8674 frames
Processed 240/8674 frames
Processed 270/8674 frames
Processed 300/8674 frames
Processed 330/8674 frames
Processed 360/8674 frames
Processed 390/8674 frames
Processed 420/8674 frames
Processed 450/8674 frames
Processed 480/8674 frames
Processed 510/8674 frames
Processed 540/8674 frames
Processed 570/8674 frames
Processed 600/8674 frames
Processed 630/8674 frames
Processed 660/8674 frames
Processed 690/8674 frames
Processed 720/8674 frames
Processed 750/8674 frames
Processed 780/8674 frames
Processed 810/8674 frames
Processed 840/8674 frames
Processed 870/8674 frames
Processed 900/8674 frames
Processed 930/8674 frames
Processed 960/8674 frames
Processed 990/8674 frames
Processed 1020/8674 frames
Processed 1050/8674 frames
Processed 1080/8674 frames
Processed 1110/8674 frames
Processed 1